# Home Credit Default Risk

## Data Cleaning

## Muhammad Darussalam
<b>dataset:</b> Home Credit Default Risk dataset | <b>Source:</b> Kaggle

Home Credit adalah perusahaan yang dibentuk pada tahun 1997 di Republik Ceko, bergerak pada jasa pinjaman secara online dan offline yang tersebar di 9 negara salah satunya Indonesia.

dalam final project ini saya akan membuat aplikasi berbasis flask-web yang akan mengeluarkan rekomendasi apakah seseorang akan kesulitan membayar pinjaman atau lancar-lancar saja


### Import Data & Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def correlation_ratio(categories, measurements):
    
    def convert(data, to):
        converted = None
        if to == 'array':
            if isinstance(data, np.ndarray):
                converted = data
            elif isinstance(data, pd.Series):
                converted = data.values
            elif isinstance(data, list):
                converted = np.array(data)
            elif isinstance(data, pd.DataFrame):
                converted = data.as_matrix()
        elif to == 'list':
            if isinstance(data, list):
                converted = data
            elif isinstance(data, pd.Series):
                converted = data.values.tolist()
            elif isinstance(data, np.ndarray):
                converted = data.tolist()
        elif to == 'dataframe':
            if isinstance(data, pd.DataFrame):
                converted = data
            elif isinstance(data, np.ndarray):
                converted = pd.DataFrame(data)
        else:
            raise ValueError("Unknown data conversion: {}".format(to))
        if converted is None:
            raise TypeError('cannot handle data conversion of type: {} to {}'.format(type(data),to))
        else:
            return converted
        
    categories = convert(categories, 'array')
    measurements = convert(measurements, 'array')
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = numerator/denominator
    return eta

In [3]:
def isna_table(df):
    isna=pd.DataFrame(df.isna().sum(),columns=['nom'])
    isna.drop(isna[isna['nom']==0].index,inplace=True)
    isna['%']=isna['nom']/len(df)*100
    return isna

In [4]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [4]:
data_train=pd.read_csv('../pake_ini.csv')

In [6]:
data_train.head()

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Active_CBC,Bad debt_CBC,Closed_CBC,Sold_CBC,CREDIT_DAY_OVERDUE,AMT_CREDIT_SUM_DEBT,Approved_PA,Canceled_PA,Refused_PA,Unused offer_PA,New_PA,Refreshed_PA,Repeater_PA,Active_PCB,Amortized debt_PCB,Approved_PCB,Canceled_PCB,Completed_PCB,Demand_PCB,Returned to the store_PCB,Signed_PCB,SK_DPD_PCB,CNT_INSTALMENT_FUTURE
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,6.0,0.0,0.0,245781.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285.0
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [7]:
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows',30)

In [5]:
data_train.drop(columns=['Unnamed: 0','SK_ID_CURR'],axis=1,inplace=True)

In [9]:
len(data_train)

307511

Dataset di atas memberikan kita 307.511 baris data riwayat peminjaman aktif (data_train), 
## Columns
### data_train:
152 entries, SK_ID_CURR to AMT_MINUS

In [10]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 144 entries, TARGET to CNT_INSTALMENT_FUTURE
dtypes: float64(88), int64(40), object(16)
memory usage: 337.8+ MB


In [11]:
pd.set_option('display.max_rows',145)
list_item=[]
for col in data_train.columns:
    list_item.append([col,data_train[col].dtype,data_train[col].isna().sum(),round((data_train[col].isna().sum()/len(data_train[col])*100),2),data_train[col].nunique(), list(data_train[col].unique()[:5])])
dfDesc_dt=pd.DataFrame(list_item,columns='feature dtype null null% unique uniquesample'.split())
dfDesc_dt

,feature,dtype,null,null%,unique,uniquesample
0,TARGET,int64,0,0.00,2,"[1, 0]"
1,NAME_CONTRACT_TYPE,object,0,0.00,2,"[Cash loans, Revolving loans]"
2,CODE_GENDER,object,0,0.00,3,"[M, F, XNA]"
3,FLAG_OWN_CAR,object,0,0.00,2,"[N, Y]"
4,FLAG_OWN_REALTY,object,0,0.00,2,"[Y, N]"
5,CNT_CHILDREN,int64,0,0.00,15,"[0, 1, 2, 3, 4]"
6,AMT_INCOME_TOTAL,float64,0,0.00,2548,"[202500.0, 270000.0, 67500.0, 135000.0, 121500.0]"
7,AMT_CREDIT,float64,0,0.00,5603,"[406597.5, 1293502.5, 135000.0, 312682.5, 5130..."
8,AMT_ANNUITY,float64,12,0.00,13672,"[24700.5, 35698.5, 6750.0, 29686.5, 21865.5]"
9,AMT_GOODS_PRICE,float64,278,0.09,1002,"[351000.0, 1129500.0, 135000.0, 297000.0, 5130..."


In [12]:
data_train.corr()['TARGET'].sort_values()

EXT_SOURCE_3                   -0.178919
EXT_SOURCE_2                   -0.160472
EXT_SOURCE_1                   -0.155317
DAYS_EMPLOYED                  -0.044932
FLOORSMAX_AVG                  -0.044003
FLOORSMAX_MEDI                 -0.043768
FLOORSMAX_MODE                 -0.043226
AMT_GOODS_PRICE                -0.039645
REGION_POPULATION_RELATIVE     -0.037227
Active_PCB                     -0.035866
ELEVATORS_AVG                  -0.034199
ELEVATORS_MEDI                 -0.033863
FLOORSMIN_AVG                  -0.033614
FLOORSMIN_MEDI                 -0.033394
LIVINGAREA_AVG                 -0.032997
LIVINGAREA_MEDI                -0.032739
FLOORSMIN_MODE                 -0.032698
TOTALAREA_MODE                 -0.032596
ELEVATORS_MODE                 -0.032131
Approved_PA                    -0.031553
Closed_CBC                     -0.030812
LIVINGAREA_MODE                -0.030685
AMT_CREDIT                     -0.030369
APARTMENTS_AVG                 -0.029498
APARTMENTS_MEDI 

## 1. FEATURE SELECTION & ENGINERING

Langkah selanjutnya adalah menentukan fitur dengan menganalisa korelasi antara semua fitur dengan fitur target

## Target FEATURE (Y)
fitur yang dilabeli sebagai target adalah TARGET yang terdapat pada data_train. dimana kolom ini menjelaskan bahwa peminjam kesulitan untuk membayar pinjaman

## Train FEATURE (X)
dalam mengukur risk berdasarkan manajemen resiko terdapat pedoman 5C antara lain:

dataset data_train memiliki informasi karakteristik pinjaman antara lain seperti:

- NAME_CONTRACT_TYPE as tipe pinjaman
- CODE_GENDER as jenis kelamin
- FLAG_OWN_CAR as apakah peminjam memiliki mobil
- FLAG_OWN_REALTY as apakah peminjam memiliki aset tidak bergerak
- CNT_CHILDREN as jumlah anak
- AMT_CREDIT as jumlah utang pinjaman
- AMT_ANNUITY as jumlah cicilan
- AMT_GOOD_PRICE as harga barang
- NAME_TYPE_SUITE as adakah yang menemani
- NAME_INCOME_TYPE as tipe pemasukan peminjam
- NAME_EDUCATION_TYPE as tingkat pendidikan peminjam
- NAME_FAMILY_STATUS as status keluarga (nikah, cerai, single)
- NAME_HOUSING_TYPE as tipe tempat tinggal
- DAYS_EMPLOYED as jumlah hari kerja
- OWN_CAR_AGE as umur kepemilikan mobil
- FLAG_CONT_MOBILE as apakah nomer HP bisa dihubungi
- OCCUPATION_TYPE as tipe pekerjaan
- CNT_FAM_MEMBERS as jumlah anggota keluarga
- ORGANIZATION_TYPE as tempat orang bekerja

In [6]:
data_dump=data_train.drop(columns=data_train.columns[43:143],axis=1)

### Feature Enginering

for simplify days_birth will be convert to age and days_employed will be convert into age_employed

In [7]:
from math import ceil,floor
data_dump['AGE']=data_dump['DAYS_BIRTH'].apply(lambda x:np.abs(ceil(x/365)) if x/365<1 else np.abs(floor(x/365)))
data_dump['YEARS_EMPLOYED']=data_dump['DAYS_EMPLOYED'].apply(lambda x:np.abs(ceil(x/365)) if x/365<1 else np.abs(floor(x/365)))
data_dump['YEARS_ID']=data_dump['DAYS_ID_PUBLISH'].apply(lambda x:np.abs(ceil(x/365)) if x/365<1 else np.abs(floor(x/365)))
data_dump['YEARS_ID_CHANGE']=data_dump['DAYS_REGISTRATION'].apply(lambda x:np.abs(ceil(x/365)) if x/365<1 else np.abs(floor(x/365)))
data_dump.drop(['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_ID_PUBLISH','DAYS_REGISTRATION'],axis=1,inplace=True)

Insert feature needed but dropped

In [8]:
l_need=['Active_CBC','Refused_PA','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','DAYS_LAST_PHONE_CHANGE','REG_CITY_NOT_WORK_CITY','New_PA','FLAG_EMP_PHONE','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','OWN_CAR_AGE','LIVE_CITY_NOT_WORK_CITY','FLAG_WORK_PHONE','CNT_CHILDREN','Canceled_PA','CNT_FAM_MEMBERS','FLAG_DOCUMENT_2','CNT_FAM_MEMBERS','Active_PCB','Approved_PA','FLAG_DOCUMENT_6','FLAG_PHONE','Completed_PCB']
for i in l_need:
    if i not in data_dump.columns:
        data_dump[i]=data_train[i]

show the last structure of feature

In [9]:
pd.set_option('display.max_rows',100)

In [10]:
list_item=[]
for col in data_dump.columns:
    list_item.append([col,data_dump[col].dtype,data_dump[col].isna().sum(),round((data_dump[col].isna().sum()/len(data_dump[col])*100),2),data_dump[col].nunique(), list(data_dump[col].unique()[:5])])
dfDesc_dp=pd.DataFrame(list_item,columns='feature dtype null null% unique uniquesample'.split())
dfDesc_dp

,feature,dtype,null,null%,unique,uniquesample
0,TARGET,int64,0,0.00,2,"[1, 0]"
1,NAME_CONTRACT_TYPE,object,0,0.00,2,"[Cash loans, Revolving loans]"
2,CODE_GENDER,object,0,0.00,3,"[M, F, XNA]"
3,FLAG_OWN_CAR,object,0,0.00,2,"[N, Y]"
4,FLAG_OWN_REALTY,object,0,0.00,2,"[Y, N]"
5,CNT_CHILDREN,int64,0,0.00,15,"[0, 1, 2, 3, 4]"
6,AMT_INCOME_TOTAL,float64,0,0.00,2548,"[202500.0, 270000.0, 67500.0, 135000.0, 121500.0]"
7,AMT_CREDIT,float64,0,0.00,5603,"[406597.5, 1293502.5, 135000.0, 312682.5, 5130..."
8,AMT_ANNUITY,float64,12,0.00,13672,"[24700.5, 35698.5, 6750.0, 29686.5, 21865.5]"
9,AMT_GOODS_PRICE,float64,278,0.09,1002,"[351000.0, 1129500.0, 135000.0, 297000.0, 5130..."


Feature Enginering days feature into years

In [11]:
data_dump['YEARS_LAST_PHONE_CHANGE']=data_dump['DAYS_LAST_PHONE_CHANGE'].dropna().apply(lambda x:np.abs(ceil(x/365)) if x/365<1 else np.abs(floor(x/365)))
data_dump.drop('DAYS_LAST_PHONE_CHANGE',axis=1,inplace=True)

Drop some feature

In [12]:
data_dump.drop(columns=['FLAG_MOBIL', 'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','FLAG_EMAIL','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START'],axis=1,inplace=True)

# Dealing with missing value

In [13]:
list_item=[]
for col in data_dump.columns:
    list_item.append([col,data_dump[col].dtype,data_dump[col].isna().sum(),round((data_dump[col].isna().sum()/len(data_dump[col])*100),2),data_dump[col].nunique(), list(data_dump[col].sample(5).drop_duplicates().values)])
dfDesc_dp=pd.DataFrame(list_item,columns='feature dtype null null% unique uniquesample'.split())
dfDesc_dp

,feature,dtype,null,null%,unique,uniquesample
0,TARGET,int64,0,0.00,2,[0]
1,NAME_CONTRACT_TYPE,object,0,0.00,2,[Cash loans]
2,CODE_GENDER,object,0,0.00,3,"[M, F]"
3,FLAG_OWN_CAR,object,0,0.00,2,"[N, Y]"
4,FLAG_OWN_REALTY,object,0,0.00,2,"[N, Y]"
5,CNT_CHILDREN,int64,0,0.00,15,"[0, 2, 1]"
6,AMT_INCOME_TOTAL,float64,0,0.00,2548,"[42867.0, 63000.0, 103500.0, 153000.0, 135000.0]"
7,AMT_CREDIT,float64,0,0.00,5603,"[276813.0, 592560.0, 254700.0, 354276.0, 72846..."
8,AMT_ANNUITY,float64,12,0.00,13672,"[17874.0, 38907.0, 6750.0, 17937.0, 6066.0]"
9,AMT_GOODS_PRICE,float64,278,0.09,1002,"[684000.0, 135000.0, 720000.0, 450000.0, 81000..."


In [14]:
null=isna_table(data_dump)
null.reset_index(inplace=True)
null=null.rename(columns={'index':'feature'})
null

,feature,nom,%
0,AMT_ANNUITY,12,0.003902
1,AMT_GOODS_PRICE,278,0.090403
2,NAME_TYPE_SUITE,1292,0.420148
3,OWN_CAR_AGE,202929,65.990810
4,CNT_FAM_MEMBERS,2,0.000650
5,EXT_SOURCE_1,173378,56.381073
6,EXT_SOURCE_2,660,0.214626
7,EXT_SOURCE_3,60965,19.825307
8,CNT_INSTALMENT_FUTURE,18067,5.875237
9,Active_CBC,44020,14.314935


### Credit Bureau Credit

In [15]:
null[(null['feature']=='Active_CBC')|(null['feature']=='Year Credit Late_Bureau')|(null['feature']=='Closed_CBC')]

,feature,nom,%
9,Active_CBC,44020,14.314935


Jumlah null di feature ini dikarenakan ketiadaan data jumlah credit aktif di credit bureau credit sehingga akan diisi 0

In [17]:
data_dump['Active_CBC'].fillna(0,inplace=True)
# data_dump['Closed_CBC'].fillna(0,inplace=True)

### Previous Application

In [18]:
null[(null['feature']=='Refused_PA')|(null['feature']=='New_PA')|(null['feature']=='Canceled_PA')|(null['feature']=='Approved_PA')]

,feature,nom,%
10,Refused_PA,16454,5.350703
11,New_PA,16454,5.350703
12,Canceled_PA,16454,5.350703
14,Approved_PA,16454,5.350703


melihat kesamaan jumlah null di atas dapat disimpulkan bahwa kasus pinjaman tersebut tidak memiliki data di Previous Application sehingga kita isi dengan nilai 0

In [19]:
data_dump['Refused_PA'].fillna(0,inplace=True)
data_dump['New_PA'].fillna(0,inplace=True)
data_dump['Canceled_PA'].fillna(0,inplace=True)
data_dump['Approved_PA'].fillna(0,inplace=True)

### Pos Cash Balanced

In [20]:
null[(null['feature']=='Completed_PCB')|(null['feature']=='Active_PCB')|(null['feature']=='CNT_INSTALMENT_FUTURE')]

,feature,nom,%
8,CNT_INSTALMENT_FUTURE,18067,5.875237
13,Active_PCB,18067,5.875237
15,Completed_PCB,18067,5.875237


melihat kesamaan jumlah null di atas dapat disimpulkan bahwa kasus pinjaman tersebut tidak memiliki data di Pos Cash Balanced sehingga kita isi dengan nilai 0

In [21]:
data_dump['Completed_PCB'].fillna(0,inplace=True)
data_dump['Active_PCB'].fillna(0,inplace=True)
data_dump['CNT_INSTALMENT_FUTURE'].fillna(0,inplace=True)

### Checking Null

In [22]:
null=isna_table(data_dump)
null.reset_index(inplace=True)
null=null.rename(columns={'index':'feature'})
null

,feature,nom,%
0,AMT_ANNUITY,12,0.003902
1,AMT_GOODS_PRICE,278,0.090403
2,NAME_TYPE_SUITE,1292,0.420148
3,OWN_CAR_AGE,202929,65.990810
4,CNT_FAM_MEMBERS,2,0.000650
5,EXT_SOURCE_1,173378,56.381073
6,EXT_SOURCE_2,660,0.214626
7,EXT_SOURCE_3,60965,19.825307
8,YEARS_LAST_PHONE_CHANGE,1,0.000325


## EXT_SOURCE Feature

EXT_SOURCE feature adalah rating yang diberikan oleh organisasi pihak ketiga terhadap seseorang/pinjaman tertentu. rating ini sudah dinormalize oleh home credit.

melihat beberapa kasus di atas, kasus Credit Bereau Credit, Previous Application, dan Pos Cash balance. dapat dilihat bahwa nan terjadi karena client/pinjaman tersebut tidak memiliki data di sana, sehingga saya berasumsi terjadinya nan pada EXT_SOURCE adalah ketiadaan data atau belum diberikan rating dari organisasi-organisasi ini kepada client/pinjaman sehingga saya memutuskan untuk membuang data-data tersebut.

In [27]:
data_dump[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

,TARGET,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,1,0.083037,0.262949,0.139376
1,0,0.311267,0.622246,NaN
2,0,NaN,0.555912,0.729567
3,0,NaN,0.650442,NaN
4,0,NaN,0.322738,NaN
...,...,...,...,...
307506,0,0.145570,0.681632,NaN
307507,0,NaN,0.115992,NaN
307508,0,0.744026,0.535722,0.218859
307509,1,NaN,0.514163,0.661024


In [32]:
data_dump[data_dump['TARGET']==1][['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

,TARGET,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,1,0.083037,0.262949,0.139376
26,1,NaN,0.548477,0.190706
40,1,NaN,0.306841,0.320163
42,1,0.468208,0.674203,0.399676
81,1,NaN,0.023952,0.720944
...,...,...,...,...
307448,1,0.073452,0.329708,0.360613
307475,1,0.634729,0.583214,0.424130
307481,1,NaN,0.713524,0.511892
307489,1,NaN,0.615261,0.397946


In [33]:
data_dump[data_dump['TARGET']==0][['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]

,TARGET,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
1,0,0.311267,0.622246,NaN
2,0,NaN,0.555912,0.729567
3,0,NaN,0.650442,NaN
4,0,NaN,0.322738,NaN
5,0,NaN,0.354225,0.621226
...,...,...,...,...
307505,0,NaN,0.346391,0.742182
307506,0,0.145570,0.681632,NaN
307507,0,NaN,0.115992,NaN
307508,0,0.744026,0.535722,0.218859


In [23]:
data_dump.drop('EXT_SOURCE_1',axis=1,inplace=True)

### Own_Car_age

In [14]:
data_dump[['FLAG_OWN_CAR','OWN_CAR_AGE']]

,FLAG_OWN_CAR,OWN_CAR_AGE
0,N,NaN
2,Y,26.0
5,N,NaN
6,Y,17.0
7,Y,8.0
...,...,...
307503,N,NaN
307505,N,NaN
307508,N,NaN
307509,N,NaN


In [24]:
data_dump['OWN_CAR_AGE'].fillna(0,inplace=True)

### Checking Again

In [25]:
null=isna_table(data_dump)
null.reset_index(inplace=True)
null=null.rename(columns={'index':'feature'})
null

,feature,nom,%
0,AMT_ANNUITY,12,0.003902
1,AMT_GOODS_PRICE,278,0.090403
2,NAME_TYPE_SUITE,1292,0.420148
3,CNT_FAM_MEMBERS,2,0.000650
4,EXT_SOURCE_2,660,0.214626
5,EXT_SOURCE_3,60965,19.825307
6,YEARS_LAST_PHONE_CHANGE,1,0.000325


membersihkan nan sisanya dengan dropping

In [26]:
data_dump.dropna(subset=null[null['%']<20]['feature'],axis=0,inplace=True)

In [27]:
list_item=[]
for col in data_dump.columns:
    list_item.append([col,data_dump[col].dtype,data_dump[col].isna().sum(),round((data_dump[col].isna().sum()/len(data_dump[col])*100),2),data_dump[col].nunique(), list(data_dump[col].sample(5).drop_duplicates().values)])
dfDesc_dp=pd.DataFrame(list_item,columns='feature dtype null null% unique uniquesample'.split())
dfDesc_dp

,feature,dtype,null,null%,unique,uniquesample
0,TARGET,int64,0,0.0,2,"[0, 1]"
1,NAME_CONTRACT_TYPE,object,0,0.0,2,"[Cash loans, Revolving loans]"
2,CODE_GENDER,object,0,0.0,3,"[M, F]"
3,FLAG_OWN_CAR,object,0,0.0,2,"[N, Y]"
4,FLAG_OWN_REALTY,object,0,0.0,2,"[N, Y]"
5,CNT_CHILDREN,int64,0,0.0,15,"[0, 1]"
6,AMT_INCOME_TOTAL,float64,0,0.0,2023,"[148500.0, 450000.0, 121500.0, 135000.0, 24750..."
7,AMT_CREDIT,float64,0,0.0,5137,"[1350000.0, 239850.0, 258768.0, 376920.0, 7819..."
8,AMT_ANNUITY,float64,0,0.0,12964,"[21433.5, 11250.0, 31630.5, 26640.0, 10116.0]"
9,AMT_GOODS_PRICE,float64,0,0.0,801,"[643500.0, 360000.0, 720000.0, 450000.0, 95400..."


unknow gender was unnecessary so, i will drop them but for XNA in organization type i will keep it consider about secret/privacy that people that won't to tell

In [28]:
data_dump.drop(index=data_dump[data_dump['CODE_GENDER'] == 'XNA'].index,inplace=True)

## Dealing with outliers

based on 2 experiment before, outliers was unnecessary attention to do something about it. so, i'll keep the outliers for this cases

# Export Data Clean

In [29]:
data_dump.to_csv('bersih.csv')